In [2]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split

# 데이터 불러오기

## 효준이 감성 점수 포함된 데이터 불러오기

In [3]:
df = pd.read_csv('서술어포함_시언감성점수.csv')

In [4]:
df

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,next_spi
0,1676.4,4.52,2.75,3.4,2069.73,1119.25,-1.635618e+08,82.5
1,1674.4,4.72,2.75,3.9,1939.30,1124.65,-1.777621e+08,83.4
2,1677.5,4.54,3.00,4.1,2106.70,1095.50,-1.980306e+08,83.9
3,1684.8,4.54,3.00,3.8,2192.36,1068.00,-1.898849e+08,84.1
4,1690.5,4.44,3.00,3.9,2142.47,1078.00,-2.402702e+08,84.4
...,...,...,...,...,...,...,...,...
115,3100.4,2.19,0.50,0.7,2326.17,1187.94,3.239718e+07,114.0
116,3115.2,2.24,0.50,1.0,2327.89,1164.65,4.511543e+07,115.6
117,3152.8,2.24,0.50,0.1,2267.15,1136.49,4.169356e+07,118.3
118,3183.5,2.25,0.50,0.6,2591.34,1109.32,4.231788e+07,121.4


In [5]:
df['sieon_sent_score'] = df['효준감성점수']

In [6]:
df.drop('효준감성점수', axis = 1, inplace = True)

In [7]:
df

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,next_spi,sieon_sent_score
0,1676.4,4.52,2.75,3.4,2069.73,1119.25,82.5,-1.635618e+08
1,1674.4,4.72,2.75,3.9,1939.30,1124.65,83.4,-1.777621e+08
2,1677.5,4.54,3.00,4.1,2106.70,1095.50,83.9,-1.980306e+08
3,1684.8,4.54,3.00,3.8,2192.36,1068.00,84.1,-1.898849e+08
4,1690.5,4.44,3.00,3.9,2142.47,1078.00,84.4,-2.402702e+08
...,...,...,...,...,...,...,...,...
115,3100.4,2.19,0.50,0.7,2326.17,1187.94,114.0,3.239718e+07
116,3115.2,2.24,0.50,1.0,2327.89,1164.65,115.6,4.511543e+07
117,3152.8,2.24,0.50,0.1,2267.15,1136.49,118.3,4.169356e+07
118,3183.5,2.25,0.50,0.6,2591.34,1109.32,121.4,4.231788e+07


In [8]:
df.to_csv('df_verb.csv')

# X,y 데이터 분할

In [7]:
X = df.copy()
y = X.pop('next_spi')

In [8]:
X

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,sieon_sent_score
0,1676.4,4.52,2.75,3.4,2069.73,1119.25,-1.635618e+08
1,1674.4,4.72,2.75,3.9,1939.30,1124.65,-1.777621e+08
2,1677.5,4.54,3.00,4.1,2106.70,1095.50,-1.980306e+08
3,1684.8,4.54,3.00,3.8,2192.36,1068.00,-1.898849e+08
4,1690.5,4.44,3.00,3.9,2142.47,1078.00,-2.402702e+08
...,...,...,...,...,...,...,...
115,3100.4,2.19,0.50,0.7,2326.17,1187.94,3.239718e+07
116,3115.2,2.24,0.50,1.0,2327.89,1164.65,4.511543e+07
117,3152.8,2.24,0.50,0.1,2267.15,1136.49,4.169356e+07
118,3183.5,2.25,0.50,0.6,2591.34,1109.32,4.231788e+07


In [9]:
y

0       82.5
1       83.4
2       83.9
3       84.1
4       84.4
       ...  
115    114.0
116    115.6
117    118.3
118    121.4
119    124.9
Name: next_spi, Length: 120, dtype: float64

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# 시언 감성 점수 Case

## 시언 감성 점수 포함 성능

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [36]:
# default parameter 
svr = SVR(kernel = 'sigmoid')

In [37]:
svr.fit(X_train, y_train)

SVR(kernel='sigmoid')

In [38]:
y_pred = svr.predict(X_test)

In [39]:
from sklearn.metrics import mean_squared_error

In [40]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [41]:
RMSE

3.5963800779679356

In [42]:
del svr, RMSE

**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- 거시경제변수만 사용 ==> 시언이 감성 점수(서술어 포함) 포함
- linear : 2.6694 ==> 2.5216
- poly : 4.9715 ==> 4.8062
- rbf : 5.8788 ==> 5.7303
- sigmoid : 3.6421 ==> 3.5963